In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [3]:
dataframe = spark.createDataFrame([
                                   (0, "sdfklsdjfk"),
                                   (1, "skdfljsdkj"),
                                   (2, "sdklfjklss"),
], ["id", "words"])
dataframe.show()

+---+----------+
| id|     words|
+---+----------+
|  0|sdfklsdjfk|
|  1|skdfljsdkj|
|  2|sdklfjklss|
+---+----------+



In [12]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/dataviz-curriculum/day_1/food.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("food.csv"), sep=",", header=True)
df.show()

Py4JJavaError: ignored

In [ ]:
# Print our schema
df.printSchema()

In [ ]:
# show the columns
df.columns

In [ ]:
# describe data
df.describe()

In [6]:
# import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [7]:
# Create the list of struct fields
schema = [StructField("food", StringType(), True), StructField("price", IntegerType(), True),]
schema

[StructField(food,StringType,true), StructField(price,IntegerType,true)]

In [8]:
# pass in our fields
final = StructType(fields=schema)
final

StructType(List(StructField(food,StringType,true),StructField(price,IntegerType,true)))

In [9]:
# now read our data with newly created schema
dataframe = spark.read.csv(SparkFiles.get('food.csv'), schema=final, sep=",", header=True)
dataframe.printSchema()

AnalysisException: ignored

In [ ]:
dataframe['price']

In [ ]:
type(dataframe['price'])

In [ ]:
dataframe.select('price')

In [ ]:
type(dataframe.select('price'))

In [ ]:
dataframe.select('price').show()

In [ ]:
# Add new column - copies 'price' column into 'newprice' column (end up with 3 col)
dataframe.withColumn('newprice', dataframe['price']).show()
# Update column name - renames price to 'newprice' (end up with 2 col)
dataframe.withColumnRenamed('price','newerprice').show()
# Double the price - doubles each price and saves under 3rd col 'doubleprice'
dataframe.withColumn('doubleprice',dataframe['price']*2).show()
# Add a dollar to the price - adds 1 to each price and saves under 3rd col
dataframe.withColumn('add_one_dollar',dataframe['price']+1).show()
# Half the price - halves each price and saves under 3rd col
dataframe.withColumn('half_price',dataframe['price']/2).show()